In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
from glob import glob

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
image_size = [100,100]

In [4]:
epochs = 5
batch_size = 25

train_path = 'C:/Users/RW565TZ/Practice/fruits-360/Training'
valid_path = 'C:/Users/RW565TZ/Practice/fruits-360/Test'

In [5]:
# Useful for getting number of files

image_files = glob(train_path + '/*/*.jp*g')
valid_image_file = glob(valid_path + '/*/*.jp*g')

In [6]:
# Useful for getting number of classes

folders = glob(train_path + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 100


In [7]:
# Add preprocessing layer
vgg = VGG16(input_shape = image_size + [3],weights='imagenet',include_top = False)

# Don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [8]:
# Our layers
x = Flatten()(vgg.output)

prediction = Dense(len(folders),activation='softmax')(x)

In [9]:
# Create model
model = Model(inputs=vgg.input,outputs=prediction)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
# Create an instance of ImageDataGenerator
gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range = 0.1,
    height_shift_range =0.1,
    shear_range = 0.1,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    preprocessing_function = preprocess_input
)

In [13]:
# Create generator

train_generator = gen.flow_from_directory(train_path,target_size=image_size,shuffle=True,batch_size=batch_size,class_mode = 'categorical')
valid_generator = gen.flow_from_directory(valid_path,target_size=image_size,shuffle=True,batch_size=batch_size,class_mode = 'categorical')

Found 49266 images belonging to 100 classes.
Found 16600 images belonging to 100 classes.


In [14]:
train_generator.class_indices

{'Apple Braeburn': 0,
 'Apple Crimson Snow': 1,
 'Apple Golden 1': 2,
 'Apple Granny Smith': 3,
 'Apple Pink Lady': 4,
 'Apple Red 1': 5,
 'Apple Red Delicious': 6,
 'Apple Red Yellow 1': 7,
 'Apple Red Yellow 2': 8,
 'Apricot': 9,
 'Avocado': 10,
 'Avocado ripe': 11,
 'Banana': 12,
 'Banana Lady Finger': 13,
 'Banana Red': 14,
 'Beetroot': 15,
 'Blueberry': 16,
 'Cactus fruit': 17,
 'Cantaloupe 1': 18,
 'Cantaloupe 2': 19,
 'Carambula': 20,
 'Cauliflower': 21,
 'Cherry 1': 22,
 'Cherry Rainier': 23,
 'Cherry Wax Black': 24,
 'Cherry Wax Red': 25,
 'Cherry Wax Yellow': 26,
 'Chestnut': 27,
 'Clementine': 28,
 'Cocos': 29,
 'Corn': 30,
 'Corn Husk': 31,
 'Cucumber Ripe': 32,
 'Dates': 33,
 'Eggplant': 34,
 'Fig': 35,
 'Ginger Root': 36,
 'Granadilla': 37,
 'Grape Blue': 38,
 'Grape Pink': 39,
 'Grape White': 40,
 'Grape White 2': 41,
 'Grapefruit Pink': 42,
 'Grapefruit White': 43,
 'Guava': 44,
 'Hazelnut': 45,
 'Huckleberry': 46,
 'Kaki': 47,
 'Kiwi': 48,
 'Kohlrabi': 49,
 'Kumquats':

In [ ]:
r = model.fit_generator(train_generator,validation_data=valid_generator,epochs=epochs,steps_per_epoch = 100,validation_steps = 300)

Epoch 1/5
100/100 [==============================] - ETA: 0s - loss: 3.1586 - accuracy: 0.3252

In [ ]:
# Plot some data

plt.plot(r.history['loss'],label='train_loss')
plt.plot(r.history['val_loss'],label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(r.history['acc'],label='train_acc')
plt.plot(r.history['val_acc'],label='val_acc')
plt.legend()
plt.show()